In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn import preprocessing
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adadelta, Adam
import matplotlib.pyplot as plt
from keras import regularizers 
import sklearn
from sklearn import svm
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import auto_regression as ar
import regular_regression as rr
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor


Using TensorFlow backend.


In [2]:
#train
#test
#load data
path = 'C:/Users/mys12/Desktop/northeastern/summer2018/data_Robust_Fingerprinting-master/DISTRIBUTED_OPENSOURCE/FINGERPRINTING_DB'

test_rss = pd.read_csv(path + '/Training_rss_21Aug17.csv', header = 0)
test_coord = pd.read_csv(path + '/Training_coordinates_21Aug17.csv', header = 0)
test_rss = test_rss.replace(100, 0)

train_rss = pd.read_csv(path + '/Test_rss_21Aug17.csv', header = 0)
train_coord = pd.read_csv(path + '/Test_coordinates_21Aug17.csv', header = 0)
train_rss = train_rss.replace(100, 0)

In [3]:
train = pd.concat([train_rss, train_coord], axis=1, ignore_index=True)
test = pd.concat([test_rss, test_coord], axis=1, ignore_index=True)
train = np.asarray(train)
test = np.asarray(test)

# first floor
train1 = train[train[:,-1]==0.0]
normalizer = preprocessing.Normalizer().fit(train1[:,:-3])
train1_r=normalizer.transform(train1[:,:-3])
train1_c=train1[:,-3:-1]
print(train1_r.shape[1])

test1 = test[test[:,-1]==0.0]
test1_r=normalizer.transform(test1[:,:-3])
test1_c=test1[:,-3:-1]

992


In [4]:
def predata(rss, locations):
    # the origin of the room
    origin = np.amin(locations,axis=0)
    #size of the room
    room_size = np.amax(locations, axis=0)-origin
    # position respect to origin
    train_Yy = locations - origin
    train_Xx = np.asarray(rss, dtype=np.float64)
    return train_Xx, train_Yy

In [5]:
# def train_val(rss, locations):
#     train_Xx, train_Yy = predata(rss, locations)
#     train_x, val_x, train_y, val_y = train_test_split(train_Xx, train_Yy, test_size=0.2)
#     return train_x, val_x, train_y, val_y

In [6]:
def accuracy(predictions, labels):
    error = np.sqrt(np.sum((predictions - labels)**2, 1))
    return error, np.mean(error)

In [7]:
trainX, trainY = predata(train1_r, train1_c)
testX, testY = predata(test1_r, test1_c)

In [8]:
clf1=rr.train_model
clf2=ar.regression

neigh = KNeighborsRegressor(n_neighbors=4)
clf3 = neigh.fit

dr = DecisionTreeRegressor(max_depth=7)
clf4 = dr.fit

er = ExtraTreeRegressor(max_depth=7)
clf5 = er.fit

clfs = [clf1,clf2,clf3,clf4,clf5]


In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=2018)

#               a = np.zeros(shape=(5,4,2))


In [10]:
def reshaped(predict):
    size = predict.shape[0]
    j = predict.reshape((2*size, 1))
    return j

In [11]:
def get_oof(clf, X_train, y_train, X_test):
#     X_train = reshaped(X_train)
#     y_train = reshaped(y_train)
#     X_test = reshaped(X_test)

    blend_train = np.zeros((y_train.shape[0],2))
    blend_test = np.zeros((X_test.shape[0],2))
    blend_test_skf = np.zeros((X_test.shape[0],2,5)) 
    
    for i, (train_index, test_index) in enumerate(list(kf.split(X_train))):
        print("Fold", i)   

        kf_X_train = X_train[train_index]
        kf_y_train = y_train[train_index]
        kf_X_test = X_train[test_index]
        kf_y_test = y_train[test_index]
        
        model = clf(kf_X_train,kf_y_train)
        
        blend_train[test_index]=model.predict(kf_X_test)  # 992*2
        
        blend_test_skf[:,:,i] = model.predict(X_test)   # 1*292*2
    
    blend_test[:,:]=blend_test_skf.mean(axis=2)
    return blend_train, blend_test

In [12]:
blend_train3, blend_test3 = get_oof(clf3, trainX, trainY, testX)


Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [39]:
blend_test3

array([[ 103.203   ,   37.66458 ],
       [  82.40365 ,   27.53373 ],
       [ 114.986   ,   37.99638 ],
       [ 132.448   ,   28.57548 ],
       [  55.4249  ,   15.64583 ],
       [  94.796   ,   53.47178 ],
       [  43.45965 ,   19.14968 ],
       [ 105.108   ,   24.80828 ],
       [  53.98    ,   16.07823 ],
       [  66.5732  ,   31.89973 ],
       [  77.25165 ,   22.00058 ],
       [  69.73575 ,   24.38083 ],
       [  67.9996  ,   23.37908 ],
       [  53.8442  ,   25.46698 ],
       [ 132.      ,   38.07608 ],
       [ 119.89    ,   69.55053 ],
       [ 133.158   ,   46.06573 ],
       [  95.0293  ,   35.93773 ],
       [ 139.5045  ,   28.26263 ],
       [  23.47205 ,   24.09053 ],
       [  59.3365  ,   38.46588 ],
       [  80.73075 ,   18.97433 ],
       [ 107.9495  ,   18.32258 ],
       [  93.3715  ,   62.86078 ],
       [ 121.742   ,   66.81118 ],
       [  50.91115 ,   24.21203 ],
       [  53.66365 ,   16.965875],
       [  99.4035  ,   37.05533 ],
       [  55.6733  ,

In [13]:
blend_train1, blend_test1 = get_oof(clf1, trainX, trainY, testX)
blend_train2, blend_test2 = get_oof(clf2, trainX, trainY, testX)
blend_train4, blend_test4 = get_oof(clf4, trainX, trainY, testX)
blend_train5, blend_test5 = get_oof(clf5, trainX, trainY, testX)

Fold 0
Train on 808 samples, validate on 203 samples
Epoch 1/100
808/808 [==============================] - 0s - loss: 4777.3464 - val_loss: 4666.5478
Epoch 2/100
808/808 [==============================] - 0s - loss: 4615.1263 - val_loss: 4374.3021
Epoch 3/100
808/808 [==============================] - 0s - loss: 4160.7385 - val_loss: 3651.5402
Epoch 4/100
808/808 [==============================] - 0s - loss: 3224.4922 - val_loss: 2428.9168
Epoch 5/100
808/808 [==============================] - 0s - loss: 2000.1538 - val_loss: 1305.3587
Epoch 6/100
808/808 [==============================] - 0s - loss: 1186.8584 - val_loss: 836.4362
Epoch 7/100
808/808 [==============================] - 0s - loss: 834.3085 - val_loss: 599.5525
Epoch 8/100
808/808 [==============================] - 0s - loss: 662.4683 - val_loss: 465.3540
Epoch 9/100
808/808 [==============================] - 0s - loss: 502.9150 - val_loss: 375.9091
Epoch 10/100
808/808 [==============================] - 0s - loss: 443.8

808/808 [==============================] - 0s - loss: 113.8526 - val_loss: 83.9688
Epoch 86/100
808/808 [==============================] - 0s - loss: 104.3741 - val_loss: 82.5099
Epoch 87/100
808/808 [==============================] - 0s - loss: 106.6708 - val_loss: 82.2566
Epoch 88/100
808/808 [==============================] - 0s - loss: 110.0533 - val_loss: 83.2057
Epoch 89/100
808/808 [==============================] - 0s - loss: 106.2696 - val_loss: 80.6440
Epoch 90/100
808/808 [==============================] - 0s - loss: 105.1382 - val_loss: 83.0844
Epoch 91/100
808/808 [==============================] - 0s - loss: 103.0856 - val_loss: 79.6526
Epoch 92/100
808/808 [==============================] - 0s - loss: 106.0414 - val_loss: 81.7282
Epoch 93/100
808/808 [==============================] - 0s - loss: 103.7085 - val_loss: 79.2273
Epoch 94/100
808/808 [==============================] - 0s - loss: 108.2645 - val_loss: 80.1057
Epoch 95/100
808/808 [==============================]

808/808 [==============================] - 0s - loss: 111.9503 - val_loss: 117.0349
Epoch 70/100
808/808 [==============================] - 0s - loss: 114.2023 - val_loss: 118.2069
Epoch 71/100
808/808 [==============================] - 0s - loss: 110.6983 - val_loss: 116.6055
Epoch 72/100
808/808 [==============================] - 0s - loss: 118.6290 - val_loss: 118.5631
Epoch 73/100
808/808 [==============================] - 0s - loss: 114.9706 - val_loss: 112.4264
Epoch 74/100
808/808 [==============================] - 0s - loss: 116.6681 - val_loss: 114.6774
Epoch 75/100
808/808 [==============================] - 0s - loss: 111.0338 - val_loss: 112.0871
Epoch 76/100
808/808 [==============================] - 0s - loss: 119.1375 - val_loss: 115.4237
Epoch 77/100
808/808 [==============================] - 0s - loss: 110.1080 - val_loss: 109.2904
Epoch 78/100
808/808 [==============================] - 0s - loss: 110.3939 - val_loss: 111.0794
Epoch 79/100
808/808 [=====================

808/808 [==============================] - 0s - loss: 122.2873 - val_loss: 98.1957
Epoch 54/100
808/808 [==============================] - 0s - loss: 119.7276 - val_loss: 96.4114
Epoch 55/100
808/808 [==============================] - 0s - loss: 122.1947 - val_loss: 97.5285
Epoch 56/100
808/808 [==============================] - 0s - loss: 117.4420 - val_loss: 96.5557
Epoch 57/100
808/808 [==============================] - 0s - loss: 122.8741 - val_loss: 96.1269
Epoch 58/100
808/808 [==============================] - 0s - loss: 127.1696 - val_loss: 95.0487
Epoch 59/100
808/808 [==============================] - 0s - loss: 115.9469 - val_loss: 94.0817
Epoch 60/100
808/808 [==============================] - 0s - loss: 119.7889 - val_loss: 93.4701
Epoch 61/100
808/808 [==============================] - 0s - loss: 121.6625 - val_loss: 92.8402
Epoch 62/100
808/808 [==============================] - 0s - loss: 115.4014 - val_loss: 92.3000
Epoch 63/100
808/808 [==============================]

808/808 [==============================] - 0s - loss: 146.1666 - val_loss: 110.1935
Epoch 38/100
808/808 [==============================] - 0s - loss: 148.4446 - val_loss: 107.7836
Epoch 39/100
808/808 [==============================] - 0s - loss: 149.1567 - val_loss: 106.4044
Epoch 40/100
808/808 [==============================] - 0s - loss: 148.0739 - val_loss: 104.8052
Epoch 41/100
808/808 [==============================] - 0s - loss: 140.3330 - val_loss: 104.8750
Epoch 42/100
808/808 [==============================] - 0s - loss: 135.9040 - val_loss: 104.2274
Epoch 43/100
808/808 [==============================] - 0s - loss: 143.3193 - val_loss: 103.8215
Epoch 44/100
808/808 [==============================] - 0s - loss: 147.1358 - val_loss: 102.4810
Epoch 45/100
808/808 [==============================] - 0s - loss: 136.0403 - val_loss: 100.6496
Epoch 46/100
808/808 [==============================] - 0s - loss: 131.0491 - val_loss: 100.9793
Epoch 47/100
808/808 [=====================

...

808/808 [==============================] - 0s - loss: 88.1820 - val_loss: 54.4839
Epoch 64/100
808/808 [==============================] - 0s - loss: 82.1089 - val_loss: 57.8097
Epoch 65/100
808/808 [==============================] - 0s - loss: 84.6270 - val_loss: 63.3674
Epoch 66/100
808/808 [==============================] - 0s - loss: 87.7086 - val_loss: 53.3698
Epoch 67/100
808/808 [==============================] - 0s - loss: 88.3616 - val_loss: 66.7927
Epoch 68/100
808/808 [==============================] - 0s - loss: 92.2728 - val_loss: 71.9750
Epoch 69/100
808/808 [==============================] - 0s - loss: 90.6536 - val_loss: 57.3318
Epoch 70/100
808/808 [==============================] - 0s - loss: 89.1288 - val_loss: 52.4110
Epoch 71/100
808/808 [==============================] - 0s - loss: 75.4277 - val_loss: 60.7301
Epoch 72/100
808/808 [==============================] - 0s - loss: 85.1730 - val_loss: 61.0227
Epoch 73/100
808/808 [==============================] - 0s - lo

808/808 [==============================] - 0s - loss: 0.0010     
Epoch 59/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 60/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 61/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 62/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 63/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 64/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 65/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 66/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 67/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 68/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 69/100
808/808 [==============================] - 0s - loss: 0.0010     
Epoch 70/100
808/808 [==============================] - 0s - loss

808/808 [==============================] - 0s - loss: 90.1416 - val_loss: 53.7484
Epoch 51/100
808/808 [==============================] - 0s - loss: 93.7834 - val_loss: 82.4424
Epoch 52/100
808/808 [==============================] - 0s - loss: 92.8407 - val_loss: 60.5760
Epoch 53/100
808/808 [==============================] - 0s - loss: 84.7342 - val_loss: 57.4620
Epoch 54/100
808/808 [==============================] - 0s - loss: 83.4781 - val_loss: 66.1764
Epoch 55/100
808/808 [==============================] - 0s - loss: 89.6587 - val_loss: 64.0963
Epoch 56/100
808/808 [==============================] - 0s - loss: 84.3119 - val_loss: 68.8674
Epoch 57/100
808/808 [==============================] - 0s - loss: 92.6157 - val_loss: 62.1461
Epoch 58/100
808/808 [==============================] - 0s - loss: 107.0049 - val_loss: 61.2380
Epoch 59/100
808/808 [==============================] - 0s - loss: 93.8040 - val_loss: 68.4291
Epoch 60/100
808/808 [==============================] - 0s - l

809/809 [==============================] - 0s - loss: 0.0010     
Epoch 44/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 45/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 46/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 47/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 48/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 49/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 50/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 51/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 52/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 53/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 54/100
809/809 [==============================] - 0s - loss: 0.0010     
Epoch 55/100
809/809 [==============================] - 0s - loss

809/809 [==============================] - 0s - loss: 105.0285 - val_loss: 107.1758
Epoch 38/100
809/809 [==============================] - 0s - loss: 96.0051 - val_loss: 100.7085
Epoch 39/100
809/809 [==============================] - 0s - loss: 97.5682 - val_loss: 91.3555
Epoch 40/100
809/809 [==============================] - 0s - loss: 97.3698 - val_loss: 91.4461
Epoch 41/100
809/809 [==============================] - 0s - loss: 102.1123 - val_loss: 77.8374
Epoch 42/100
809/809 [==============================] - 0s - loss: 98.2447 - val_loss: 86.8086
Epoch 43/100
809/809 [==============================] - 0s - loss: 107.5499 - val_loss: 95.7518
Epoch 44/100
809/809 [==============================] - 0s - loss: 104.0446 - val_loss: 91.6536
Epoch 45/100
809/809 [==============================] - 0s - loss: 104.1071 - val_loss: 112.8939
Epoch 46/100
809/809 [==============================] - 0s - loss: 95.5077 - val_loss: 96.0252
Epoch 47/100
809/809 [==============================] -

In [14]:
blend_train1 =reshaped(blend_train1)
blend_test1 = reshaped(blend_test1)
blend_train2 =reshaped(blend_train2)
blend_test2 = reshaped(blend_test2)
blend_train3 =reshaped(blend_train3)
blend_test3 = reshaped(blend_test3)
blend_train4 =reshaped(blend_train4)
blend_test4 = reshaped(blend_test4)
blend_train5 =reshaped(blend_train5)
blend_test5 = reshaped(blend_test5)

In [17]:
blend_trainX = np.column_stack((blend_train1,blend_train2, blend_train3,blend_train4,blend_train5))
blend_testX = np.column_stack((blend_test1, blend_test2, blend_test3,blend_test4,blend_test5))

In [18]:
trainY_s = reshaped(trainY)
testY_s = reshaped(testY)

In [19]:
# stacking model 1
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(blend_trainX, trainY_s)
pred1 = lr.predict(blend_testX)
e, a = accuracy(pred1.reshape((226,2)), testY)
print(a)

8.15774421613


In [27]:
# stacking model 3
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=4)
neigh.fit(blend_trainX, trainY_s)
pred3 = neigh.predict(blend_testX)
e3, a3 = accuracy(pred3.reshape((226,2)), testY)
print(a3)

8.85795593186


In [56]:

for j, clf in enumerate(clfs):    
    print(j, clf)
    blend_test_j = np.zeros(shape=(5, len(testX),2))
    for i, (train_index, test_index) in enumerate(skf):
        print("Fold", i)   

        X_train = trainX[train_index]
        y_train = trainY[train_index]
        X_test = trainX[test_index]
        y_test = trainY[test_index]

    #     model1 = rr.train_model(X_train,y_train)
        model = clf(X_train,y_train)
        blend_train[j, test_index, :] =  model.predict(X_test)
        blend_test_j[:,:,i] =   model.predict(testX)
        

    blend_test[:,:,j] = blend_test_j.mean(axis=2)

0 <function train_model at 0x0000016E1020A400>
Fold 0
Train on 808 samples, validate on 203 samples
Epoch 1/100
808/808 [==============================] - 0s - loss: 4772.4495 - val_loss: 4640.5937
Epoch 2/100
808/808 [==============================] - 0s - loss: 4580.5283 - val_loss: 4301.8972
Epoch 3/100
808/808 [==============================] - 0s - loss: 4048.7716 - val_loss: 3501.5784
Epoch 4/100
808/808 [==============================] - 0s - loss: 3026.3709 - val_loss: 2262.0923
Epoch 5/100
808/808 [==============================] - 0s - loss: 1826.8906 - val_loss: 1311.9731
Epoch 6/100
808/808 [==============================] - 0s - loss: 1118.8428 - val_loss: 884.2793
Epoch 7/100
808/808 [==============================] - 0s - loss: 783.5000 - val_loss: 644.1486
Epoch 8/100
808/808 [==============================] - 0s - loss: 613.6579 - val_loss: 505.8835
Epoch 9/100
808/808 [==============================] - 0s - loss: 510.0662 - val_loss: 413.3073
Epoch 10/100
808/808 [===

808/808 [==============================] - 0s - loss: 112.1969 - val_loss: 92.3370
Epoch 85/100
808/808 [==============================] - 0s - loss: 108.2908 - val_loss: 90.4180
Epoch 86/100
808/808 [==============================] - 0s - loss: 108.6955 - val_loss: 90.5200
Epoch 87/100
808/808 [==============================] - 0s - loss: 107.2822 - val_loss: 89.8054
Epoch 88/100
808/808 [==============================] - 0s - loss: 109.6533 - val_loss: 89.9213
Epoch 89/100
808/808 [==============================] - 0s - loss: 102.1250 - val_loss: 89.6816
Epoch 90/100
808/808 [==============================] - 0s - loss: 103.7372 - val_loss: 91.9743
Epoch 91/100
808/808 [==============================] - 0s - loss: 105.7120 - val_loss: 87.9380
Epoch 92/100
808/808 [==============================] - 0s - loss: 106.4221 - val_loss: 91.4754
Epoch 93/100
808/808 [==============================] - 0s - loss: 103.8876 - val_loss: 86.8150
Epoch 94/100
808/808 [==============================]

808/808 [==============================] - 0s - loss: 116.0455 - val_loss: 96.9169
Epoch 69/100
808/808 [==============================] - 0s - loss: 110.3638 - val_loss: 96.5894
Epoch 70/100
808/808 [==============================] - 0s - loss: 119.3247 - val_loss: 94.0773
Epoch 71/100
808/808 [==============================] - 0s - loss: 113.0894 - val_loss: 93.1979
Epoch 72/100
808/808 [==============================] - 0s - loss: 114.1282 - val_loss: 93.8057
Epoch 73/100
808/808 [==============================] - 0s - loss: 115.5467 - val_loss: 91.9086
Epoch 74/100
808/808 [==============================] - 0s - loss: 114.4644 - val_loss: 92.6957
Epoch 75/100
808/808 [==============================] - 0s - loss: 115.7478 - val_loss: 91.9154
Epoch 76/100
808/808 [==============================] - 0s - loss: 111.0712 - val_loss: 92.5960
Epoch 77/100
808/808 [==============================] - 0s - loss: 113.1139 - val_loss: 92.5831
Epoch 78/100
808/808 [==============================]

808/808 [==============================] - 0s - loss: 130.7512 - val_loss: 115.7854
Epoch 53/100
808/808 [==============================] - 0s - loss: 120.5768 - val_loss: 117.2649
Epoch 54/100
808/808 [==============================] - 0s - loss: 128.6807 - val_loss: 115.8351
Epoch 55/100
808/808 [==============================] - 0s - loss: 127.7445 - val_loss: 113.6157
Epoch 56/100
808/808 [==============================] - 0s - loss: 130.4411 - val_loss: 114.6029
Epoch 57/100
808/808 [==============================] - 0s - loss: 124.0644 - val_loss: 114.1236
Epoch 58/100
808/808 [==============================] - 0s - loss: 120.8488 - val_loss: 112.9613
Epoch 59/100
808/808 [==============================] - 0s - loss: 119.9951 - val_loss: 112.3705
Epoch 60/100
808/808 [==============================] - 0s - loss: 127.2782 - val_loss: 113.8438
Epoch 61/100
808/808 [==============================] - 0s - loss: 122.1985 - val_loss: 114.3501
Epoch 62/100
808/808 [=====================

808/808 [==============================] - 0s - loss: 157.6064 - val_loss: 122.5402
Epoch 36/100
808/808 [==============================] - 0s - loss: 151.4503 - val_loss: 117.0512
Epoch 37/100
808/808 [==============================] - 0s - loss: 143.3432 - val_loss: 116.4519
Epoch 38/100
808/808 [==============================] - 0s - loss: 145.5523 - val_loss: 115.8898
Epoch 39/100
808/808 [==============================] - 0s - loss: 152.8682 - val_loss: 114.5363
Epoch 40/100
808/808 [==============================] - 0s - loss: 152.9077 - val_loss: 111.6395
Epoch 41/100
808/808 [==============================] - 0s - loss: 144.2571 - val_loss: 112.1791
Epoch 42/100
808/808 [==============================] - 0s - loss: 142.9322 - val_loss: 110.6159
Epoch 43/100
808/808 [==============================] - 0s - loss: 141.3125 - val_loss: 110.0932
Epoch 44/100
808/808 [==============================] - 0s - loss: 135.3025 - val_loss: 109.7531
Epoch 45/100
808/808 [=====================

809/809 [==============================] - 0s - loss: 269.3947 - val_loss: 181.6825
Epoch 20/100
809/809 [==============================] - 0s - loss: 266.2401 - val_loss: 172.6825
Epoch 21/100
809/809 [==============================] - 0s - loss: 246.9815 - val_loss: 167.5511
Epoch 22/100
809/809 [==============================] - 0s - loss: 237.9934 - val_loss: 156.6410
Epoch 23/100
809/809 [==============================] - 0s - loss: 227.8983 - val_loss: 147.0431
Epoch 24/100
809/809 [==============================] - 0s - loss: 209.9623 - val_loss: 137.0303
Epoch 25/100
809/809 [==============================] - 0s - loss: 202.8689 - val_loss: 129.8829
Epoch 26/100
809/809 [==============================] - 0s - loss: 208.6081 - val_loss: 120.9383
Epoch 27/100
809/809 [==============================] - 0s - loss: 193.0405 - val_loss: 115.2023
Epoch 28/100
809/809 [==============================] - 0s - loss: 185.4166 - val_loss: 111.6078
Epoch 29/100
809/809 [=====================

In [58]:
blend_train[69]

array([[ 49.99036026,   0.        ,   0.        ,   0.        ,   0.        ],
       [ 34.67944336,   0.        ,   0.        ,   0.        ,   0.        ]])

In [59]:
# stacking model 3
from sklearn.neighbors import KNeighborsRegressor
neig = KNeighborsRegressor(n_neighbors=4)
neig.fit(blend_train,trainY)
pred3 = neig.predict(blend_test)
e3, a3 = accuracy(pred3, test_y)
print(a3)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [77]:
a = np.zeros((3,2,5))
print(a)
print(a[:,:,3])

[[[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]]
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [71]:

np.array([[2,0],[1,9],[3,3]])

array([[2, 0],
       [1, 9],
       [3, 3]])